# 资金去向非嫌疑人

In [285]:
import pymysql as ps
import re
import xlrd
import pandas as pd
import numpy as np
#连接数据库
db_host = 'localhost'
db_user = 'root'
db_pwd = '521521'
db_name = 'sxzy'
char_type = 'utf-8'
db = ps.connect(db_host, db_user, db_pwd, db_name, charset='utf8')
couser = db.cursor()

In [286]:
# 获取嫌疑人名单
sql='select jymc from person_info'
couser.execute(sql)
rows = couser.fetchall()
jymc_list=[i[0] for i in rows]

In [162]:
# 获取除基金理财外数据情况
sql="select distinct   cxkh,jymc,jdbz,jysj,jyje,jyye,jydfzkh,jydfmc from gas_bank_records where jdbz='出'\
AND jydfmc NOT LIKE '%%财付通%%'AND jydfmc NOT LIKE '%%支付宝%%' AND jydfmc NOT LIKE '%%待清算电子交换%%'  \
AND jydfmc NOT LIKE '%%待清算电子交换%%' AND jydfmc NOT LIKE '%%保险事业所%%'AND jydfmc NOT LIKE '%%微众银行%%'\
AND jydfmc NOT LIKE '%%浙江网商银行%%' AND jydfmc NOT LIKE '%%备付金%%' AND jydfmc NOT LIKE '%%国家统计局%%'\
AND jydfmc NOT LIKE '%%开放式%%' AND jydfmc NOT LIKE '%%基金%%'AND jydfmc NOT LIKE '%%理财%%'\
AND jydfmc NOT LIKE '%%非凡资产%%' AND jydfmc NOT LIKE '%%泰康资产%%'AND jydfmc NOT LIKE '%%待清算%%'\
AND jydfmc NOT LIKE '%%批量代付业务%%' AND jydfmc NOT LIKE '%%吕梁市农机%%'AND jydfmc NOT LIKE '%%临时存欠%%'\
AND jydfmc NOT LIKE '%%保险%%' AND jydfmc NOT LIKE '%%吕梁市泰化学校%%'AND jydfmc NOT LIKE '%%吕梁市离石区永宁小学校%%'\
AND jydfmc NOT LIKE '%%西南政法大学%%' AND jydfmc NOT LIKE '%%中国银联%%'AND jydfmc NOT LIKE '%%其他代理业务资金%%'\
AND jydfmc NOT LIKE '%%招联消费金融%%' AND jydfmc NOT LIKE '%%广州银联%%'AND jydfmc NOT LIKE '%%吕梁市离石区永宁小学校%%'\
AND jydfmc NOT LIKE '%%汇出汇款%%' AND jydfmc NOT LIKE '%%银联商务商户%%'AND jydfmc NOT LIKE '%%山西省肿瘤医院%%'\
AND jydfmc NOT LIKE '%%通联支付%%' AND jydfmc NOT LIKE '%%证券%%'AND jydfmc NOT LIKE '%%银行%%'\
AND jydfmc NOT LIKE '%%中邮消费金融%%' AND jydfmc NOT LIKE '%%钱袋宝%%'AND jydfmc NOT LIKE '%%银联商务%%'\
AND jydfmc NOT LIKE '%%南方天天利A%%' AND jydfmc NOT LIKE '%%应付个人活期储蓄%%'AND jydfmc NOT LIKE '%%特约%%'\
AND jydfmc NOT LIKE '%%企业钱包%%' AND jydfmc NOT LIKE '%%服务费收入户%%'AND jydfmc NOT LIKE '%%乾元%%'\
AND jydfmc NOT LIKE '%%高明明 先生%%' AND jydfmc NOT LIKE '%%QQ转账%%'AND jydfmc NOT LIKE '%%服务费清算%%'\
AND jydfmc NOT LIKE '%%李叶 女士%%' AND jydfmc NOT LIKE '%%支付平台%%'AND jydfmc NOT LIKE '%%手续费收入户%%'\
AND jydfmc NOT LIKE '%%银联代付%%' AND jydfmc NOT LIKE '%%德克士%%'AND jydfmc NOT LIKE '%%美团%%'\
AND jydfmc NOT LIKE '%%南方通利%%' AND jydfmc NOT LIKE '%%美团%%'AND jydfmc NOT LIKE '%%特约%%'\
AND jydfmc NOT LIKE '%%滴滴%%' AND jydfmc NOT LIKE '%%同程%%'AND jydfmc NOT LIKE '%%特约%%'\
AND jydfmc NOT LIKE '%%淘宝%%' AND jydfmc NOT LIKE '%%银联代付%%'AND jydfmc NOT LIKE '%%特约%%'\
AND JYDFMC NOT LIKE '%%微信%%' AND jydfmc NOT LIKE '%%参与银联%%'"

In [163]:
couser.execute(sql)
rows = couser.fetchall()

In [164]:
df_columns=['cxkh','jymc','jdbz','jysj','jyje','jyye','jydfzkh','jydfmc']
df=pd.DataFrame(np.array(rows),columns=df_columns)

In [165]:
df['jysj']=pd.to_datetime(df['jysj'],format='%Y-%m-%d %H:%M:%S')
df['jyje']=df['jyje'].astype(str).astype(float)

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388407 entries, 0 to 388406
Data columns (total 8 columns):
cxkh       388407 non-null object
jymc       384327 non-null object
jdbz       388407 non-null object
jysj       388407 non-null datetime64[ns]
jyje       388407 non-null float64
jyye       388407 non-null object
jydfzkh    385922 non-null object
jydfmc     388407 non-null object
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 23.7+ MB


In [169]:
#数据填充,目的是为了分组,防止数据缺失
df=df.fillna('weizhi')

In [170]:
import numpy as np
df_go=pd.pivot_table(df,index=['jymc','cxkh','jydfmc','jydfzkh'],aggfunc={'jyje':[sum,len],'jysj':[max,min]})
df_go_1=df_go.reset_index()
df_go_1.columns=['jymc','cxkh','jydfmc','jydfzkh','czcs','czje','last_time','first_time']


jyje  \
                                                                    len   
jymc   cxkh                jydfmc           jydfzkh                       
weizhi 153823102           中阳县瑞昶煤化有限公司      146734766606            2.0   
                           临县时通运业有限公司       35810130000010150       1.0   
                           单笔转账汇款(大、小额)     weizhi                 45.0   
                           吕梁创想商贸有限公司       358900125510703         1.0   
                           吕梁华飞能源有限公司       2005300000000179        5.0   
                           吕梁华龙煤业有限公司       155538964               1.0   
                           吕梁市华昊物流有限公司      161301201020536623      4.0   
                           吕梁泰荣运业有限公司       543000331012017019019   2.0   
                           吕梁浩弘商贸有限公司       400930843               8.0   
                           吕梁滋盛运输有限公司       14050169690800000545    5.0   
                           吕梁百旺金赋科技有限公司     543000001018010602244   1.0   
                           国家金库吕梁市离石区支库     2560                   11.0   
                           山西东江煤业集团有限公司     553103010300000027360   2.0   
                           山西励申源贸易有限公司      0509025509020218978     1.0   
                           山西厚祥精煤有限公司       648001040008886         1.0   
                           山西厚翔精煤有限公司       648001040008886         1.0   
                           山西方山金晖凯川煤业有限公司   553151010300000009086   2.0   
                           山西正晖能源有限公司       146749765583            1.0   
                           山西联众物华商贸有限公司     04645201040003985       3.0   
                                            600878936               3.0   
                           山西顺驰路捷汽车销售服务有限公司 7261210182200016645     2.0   
                           方山县庞泉煤焦有限公司      14001697208052503013    1.0   
                           方山县德润煤焦有限公司      04-641001040013453      1.0   
                                            641001040013453         2.0   
                           晋能大土河热电有限公司      697643451               1.0   
                           杭州信雅达科技有限公司      1202021119900091268     1.0   
                           汾阳市恒盛源煤业有限公司     699613546               5.0   
                           汾阳市盛渊煤业有限公司      141709142652            5.0   
                           高明明              6226223903035320       26.0   
       158837542           乔宇飞              6226223900615546        9.0   
...                                                                 ...   
黄钦圳    6222031408002507186 邵宇               6230580000172878949     1.0   
                           郑加能              6214837603162763        1.0   
                           郑森               6222620780003697063     2.0   
                                            6222620780003697068     1.0   
                           郝义省              6224216666667894420     1.0   
                                            6230580000235936346     1.0   
                           郭旭锋              6228413034520611378     1.0   
                           郭海标              6212881805000003526     1.0   
                           鄢小美              6217867000001598651     1.0   
                           鄢能文              6214662940070861        1.0   
                           钟航兵              6228460088015331670     1.0   
                           钱茜               6228480439937077170     1.0   
                           问建军              4100623582488888        1.0   
                           阮润坤              6217003110019623683     1.0   
                           陈世辉              6222600710007237755     1.0   
                           陈丽妍              6217857000042984276     1.0   
                           陈庭漫              6222620710029730700     1.0   
                           陈楚强              6212252019000906917     1.0   
                           马买艳              6236681420023549793     3.0   
                           马文成              6217000860005759429     1.0   
                 

In [179]:
#s筛选去向为非嫌疑人的数据
df_go_2=df_go_1[df_go_1['jydfmc'].isin(jymc_list)==False]
df_go_2.head()

,jymc,cxkh,jydfmc,jydfzkh,czcs,czje,last_time,first_time
0,weizhi,153823102,中阳县瑞昶煤化有限公司,146734766606,2.0,11000000.00,2019-08-13 10:08:56,2019-08-13 10:03:49
1,weizhi,153823102,临县时通运业有限公司,35810130000010150,1.0,502422.00,2018-01-23 16:27:59,2018-01-23 16:27:59
2,weizhi,153823102,单笔转账汇款(大、小额),weizhi,45.0,1474.68,2019-10-09 09:17:40,2018-01-03 16:58:33
3,weizhi,153823102,吕梁创想商贸有限公司,358900125510703,1.0,5500.00,2019-06-14 16:46:03,2019-06-14 16:46:03
4,weizhi,153823102,吕梁华飞能源有限公司,2005300000000179,5.0,23900000.00,2019-10-09 09:17:40,2019-03-11 09:38:15


In [181]:
def judgement_level(x):
    if 100000<= x<500000:
        return 100000
    elif 500000<=x <1000000: 
        return  500000
    elif 1000000<=x<5000000:
        return 1000000
    elif 5000000<=x<10000000:
        return 5000000
    elif x>=10000000:
        return 10000000
    else :
        return 0        

In [182]:
df_go_2['level']=df_go_2['czje'].apply(lambda x:judgement_level(x))

D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [183]:
#输出不同等级的数据
for i in set(df_go_2['level']):
    df_out=df_go_2[df_go_2['level']==i]
    df_out.to_excel('c:/users/user1/desktop/中阳需求/'+str(i)+".xlsx",index=False,encoding='utf-8')

In [184]:
#输出总的等级数据
df_go_2.to_excel('c:/users/user1/desktop/中阳需求/总的出账情况输出.xlsx',index=False,encoding='utf-8')

# 理财类数据


In [273]:
sql_lc="SELECT  distinct cxkh,jymc,jysj,jdbz,jyje,jyye,jydfmc,jydfzkh,beiz,zysm  FROM gas_bank_records\
 WHERE jysj>='2019-01-01' and (\
    beiz LIKE '%%理财%%' OR zysm LIKE '%%理财%%' OR jydfmc LIKE '%%理财%%' \
    OR beiz LIKE '%%证券%%' OR zysm LIKE '%%证券%%' OR jydfmc LIKE '%%证券%%' \
    OR beiz LIKE '%%保险%%' OR zysm LIKE '%%保险%%' OR jydfmc LIKE '%%保险%%' \
    OR beiz LIKE '%%非凡资产%%' OR zysm LIKE '%%非凡资产%%' OR jydfmc LIKE '%%非凡资产%%' \
    OR beiz LIKE '%%基金%%' OR zysm LIKE '%%基金%%' OR jydfmc LIKE '%%基金%%' \
    OR beiz LIKE '%%乾元%%' OR zysm LIKE '乾元' OR jydfmc LIKE '%%乾元%%'\
    OR beiz LIKE '%%泰康%%' OR zysm LIKE '泰康' OR jydfmc LIKE '%%泰康%%')"

In [274]:
couser.execute(sql_lc)
rows = couser.fetchall()
df_columns=[ 'cxkh','jymc','jysj','jdbz','jyje',"jyye","jydfmc","jydfzkh",'beiz','zysm']
df_lc=pd.DataFrame(np.array(rows),columns=df_columns)
df_lc['jysj']=pd.to_datetime(df_lc['jysj'],format='%Y-%m-%d %H:%M:%S')
df_lc['jyje']=df_lc['jyje'].astype(str).astype(float)


In [275]:
df_lc=df_lc.fillna('weizhi')

In [276]:
#备注为“该账户可能办理我行理财产品，余额请以账户余额为主”
df_li_bz=df_lc[df_lc['beiz'].str.contains(r'.*?该账户可能办理我行理财产品，余额请以账户余额为主*')]
df_li_bz.to_excel('c:/users/user1/desktop/中阳需求/理财——备注为该账户可能办理我行理财产品，余额请以账户余额为主.xlsx',encoding='utf-8',index=False)

In [288]:
#筛选部分特殊数据，主要是理财数据
df_li_other=df_lc[(df_lc['beiz'].str.contains(r'.*?该账户可能办理我行理财产品，余额请以账户余额为主*')==False)&(df_lc['beiz']!='工资')\
                 &(df_lc['beiz']!='代发工资')&(df_lc['beiz'].str.contains(r'.*?差旅费|养老金*')==False)]


In [278]:
df_li_bz.to_excel('c:/users/user1/desktop/中阳需求/理财——备注为该账户可能办理我行理财产品，余额请以账户余额为主.xlsx',encoding='utf-8',index=False)

In [289]:
df_li_other.shape

(1161, 10)

In [280]:
df_li_other_level=pd.pivot_table(df_li_other,index=['cxkh','jymc'],columns='jdbz',aggfunc={'jyje':sum},fill_value=0).reset_index()

In [281]:
#字段名的拼接
df_li_other_level.columns=[s1+'_'+s2 for s1,s2 in df_li_other_level.columns.tolist()]

In [282]:
#非该账户数据疑似办理理财产品最大最小时间统计。
df_li_other_time=pd.pivot_table(df_li_other,index=['cxkh','jymc'],aggfunc={'jysj':[max,min]}).reset_index()
df_li_other_time.columns=[s1+'_'+s2 for s1,s2 in df_li_other_time.columns.tolist()]

In [283]:
# 数据更新
df_li_other_level_fin=pd.merge(df_li_other_level,df_li_other_time,left_on='cxkh_',right_on='cxkh_')
df_li_other_level_fin=df_li_other_level_fin[["cxkh_","jymc__x","jyje_出","jyje_进","jysj_max","jysj_min"]]
df_li_other_level_fin.columns=['交易卡号','交易名称','出账金额','进账金额','最后一次交易金额','最早一次交易金额']
df_li_other_level_fin['角色']=df_li_other_level_fin['交易名称'].apply(lambda x : '嫌疑人/亲属' if x in jymc_list else '非嫌疑人/亲属')

In [284]:
df_li_other_level_fin.head()

,交易卡号,交易名称,出账金额,进账金额,最后一次交易金额,最早一次交易金额,角色
0,0509025501203896208,崔吉平,1274.00,0.00,2019-07-15 13:17:25,2019-07-11 14:59:55,嫌疑人/亲属
1,140482588095,薛艳峰,26386.92,0.00,2019-10-14 14:59:11,2019-06-03 17:08:29,嫌疑人/亲属
2,140484706551,马晓强,75174.00,49264.00,2019-11-11 11:28:34,2019-01-03 09:53:27,非嫌疑人/亲属
3,141726457061,郭凤兰,50543.96,45623.66,2019-10-19 08:07:28,2019-01-14 20:54:18,嫌疑人/亲属
4,144240653073,李艳林,2290.00,590.44,2019-11-22 21:36:09,2019-04-24 15:46:04,非嫌疑人/亲属


In [272]:
#数据楚竹
writer=pd.ExcelWriter('c:/users/user1/desktop/中阳需求/理财数据情况说明.xlsx')

df_li_other.to_excel(writer,sheet_name='理财类整体数据',encoding='utf-8',index=False)
df_li_other_level_fin.to_excel(writer,sheet_name='理财类数据统计情况',encoding='utf-8',index=False)
df_li_bz.to_excel(writer,sheet_name='备注为疑似该账户理财的信息',encoding='utf-8',index=False)
writer.save()
writer.close()


In [291]:
df_li_bz.info() #获取数据缺失值情况。

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11175 entries, 188 to 12161
Data columns (total 10 columns):
cxkh       11175 non-null object
jymc       11175 non-null object
jysj       11175 non-null datetime64[ns]
jdbz       11175 non-null object
jyje       11175 non-null float64
jyye       11175 non-null object
jydfmc     11175 non-null object
jydfzkh    11175 non-null object
beiz       11175 non-null object
zysm       11175 non-null object
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 960.4+ KB


In [304]:
df_li_bz_level=pd.pivot_table(df_li_bz,index=['cxkh','jymc'],columns='jdbz',aggfunc={'jyje':sum},fill_value=0).reset_index()
df_li_bz_level.head()

cxkh    jymc         jyje             
jdbz                                      出            进
0            153823102  weizhi  64866608.23  62081115.79
1            158837542  weizhi    947711.00    961072.92
2         390100001055     芦继香         0.00        19.98
3     3903014430007656     王瑞花       345.23         0.00
4     4720680108081506     鲁立国         0.00       128.40

In [305]:
df_li_bz_level.columns=[s1+'_'+s2 for s1,s2 in df_li_bz_level.columns.tolist()]
df_li_bz_level.head()
df_li_bz_time=pd.pivot_table(df_li_bz,index=['cxkh','jymc'],aggfunc={'jysj':[max,min]}).reset_index()
df_li_bz_time.columns=[s1+'_'+s2 for s1,s2 in df_li_bz_time.columns.tolist()]
# 数据更新
df_li_bz_level_fin=pd.merge(df_li_bz_level,df_li_bz_time,left_on='cxkh_',right_on='cxkh_')
df_li_bz_level_fin=df_li_bz_level_fin[["cxkh_","jymc__x","jyje_出","jyje_进","jysj_max","jysj_min"]]
df_li_bz_level_fin.columns=['交易卡号','交易名称','出账金额','进账金额','最后一次交易时间','最早一次交易时间']
df_li_bz_level_fin['角色']=df_li_other_level_fin['交易名称'].apply(lambda x : '嫌疑人/亲属' if x in jymc_list else '非嫌疑人/亲属')
df_li_bz_level_fin.head()


,交易卡号,交易名称,出账金额,进账金额,最后一次交易时间,最早一次交易时间,角色
0,153823102,weizhi,64866608.23,62081115.79,2019-12-31 13:20:34,2019-01-11 15:17:13,嫌疑人/亲属
1,158837542,weizhi,947711.00,961072.92,2020-01-03 17:10:30,2019-05-22 12:26:57,嫌疑人/亲属
2,390100001055,芦继香,0.00,19.98,2019-09-21 01:46:25,2019-03-21 01:55:20,非嫌疑人/亲属
3,3903014430007656,王瑞花,345.23,0.00,2019-12-15 02:00:46,2019-12-15 02:00:46,嫌疑人/亲属
4,4720680108081506,鲁立国,0.00,128.40,2019-12-21 01:26:08,2019-03-21 01:29:21,非嫌疑人/亲属


In [301]:
df_li_bz_level_fin.columns

Index(['cxkh_', 'jymc__x', 'jyje_出', 'jyje_进', 'jymc__y', 'jysj_max',
       'jysj_min'],
      dtype='object')

In [306]:
df_li_bz_level_fin.shape

(60, 7)

In [313]:
df_li_bz_level_fin[df_li_bz_level_fin['交易卡号'].isin(set(df_li_other_level_fin['交易卡号']))]

,交易卡号,交易名称,出账金额,进账金额,最后一次交易时间,最早一次交易时间,角色
5,4720683900496666,陈月娥,250032.43,250032.43,2019-04-14 10:22:44,2019-01-23 11:16:31,嫌疑人/亲属
6,4720683943556666,陈月娥,250032.43,250032.43,2019-04-14 10:22:44,2019-01-23 11:16:31,嫌疑人/亲属
11,6216913900017881,刘桂珍,196869.80,197041.04,2019-11-28 17:43:34,2019-01-03 18:57:21,非嫌疑人/亲属
12,6216913900145856,弓卫红,6662566.25,7735861.51,2020-01-17 18:22:48,2019-01-02 00:38:56,非嫌疑人/亲属
13,6216913901295049,毕丹丹,8289.94,8289.94,2019-10-22 06:27:40,2019-03-15 22:22:25,非嫌疑人/亲属
17,6226170101187595,解丽华,2120736.74,2104863.05,2019-12-21 01:00:32,2019-01-01 12:10:06,非嫌疑人/亲属
18,6226180992288881,郑婷婷,610002.03,620481.17,2020-01-07 05:16:58,2019-01-15 14:22:14,非嫌疑人/亲属
19,6226183900900001,陈月娥,82302914.19,79524618.87,2020-01-17 11:03:35,2019-01-01 00:23:56,非嫌疑人/亲属
21,6226190900991997,官侠,7751583.96,7476155.89,2020-01-14 19:47:53,2019-01-02 19:14:35,非嫌疑人/亲属
22,6226193900031475,武利军,320.36,135.01,2019-04-06 05:36:31,2019-01-11 03:48:31,非嫌疑人/亲属


In [315]:
df_li_bz_level_fin[df_li_bz_level_fin['交易卡号'].isin(df_li_other_level_fin['交易卡号'])==False].head()

,交易卡号,交易名称,出账金额,进账金额,最后一次交易时间,最早一次交易时间,角色
0,153823102,weizhi,6.486661e+07,6.208112e+07,2019-12-31 13:20:34,2019-01-11 15:17:13,嫌疑人/亲属
1,158837542,weizhi,9.477110e+05,9.610729e+05,2020-01-03 17:10:30,2019-05-22 12:26:57,嫌疑人/亲属
2,390100001055,芦继香,0.000000e+00,1.998000e+01,2019-09-21 01:46:25,2019-03-21 01:55:20,非嫌疑人/亲属
3,3903014430007656,王瑞花,3.452300e+02,0.000000e+00,2019-12-15 02:00:46,2019-12-15 02:00:46,嫌疑人/亲属
4,4720680108081506,鲁立国,0.000000e+00,1.284000e+02,2019-12-21 01:26:08,2019-03-21 01:29:21,非嫌疑人/亲属
7,4720683948022227,任志强,1.972380e+03,4.090000e+00,2019-09-21 01:09:43,2019-03-21 01:11:15,非嫌疑人/亲属
8,6216912304489936,陈春梅,2.160000e+01,1.200000e-01,2020-01-11 07:02:30,2019-02-17 06:20:52,非嫌疑人/亲属
9,6216912305100037,吴文琛,1.620000e+01,3.000000e-02,2019-11-18 06:37:22,2019-03-15 06:16:18,非嫌疑人/亲属
10,6216912907487519,刘春竹,2.322506e+07,2.304458e+07,2019-12-21 01:28:18,2019-10-08 19:48:24,非嫌疑人/亲属
14,6216920019575642,王云秀,5.053277e+05,5.053277e+05,2019-11-27 00:02:11,2019-01-11 11:38:02,嫌疑人/亲属


In [322]:
df_li_bz_level_fin[df_li_bz_level_fin['交易卡号'].isin(df_li_other_level_fin['交易卡号'])==True]

,交易卡号,交易名称,出账金额,进账金额,最后一次交易时间,最早一次交易时间,角色
5,4720683900496666,陈月娥,250032.43,250032.43,2019-04-14 10:22:44,2019-01-23 11:16:31,嫌疑人/亲属
6,4720683943556666,陈月娥,250032.43,250032.43,2019-04-14 10:22:44,2019-01-23 11:16:31,嫌疑人/亲属
11,6216913900017881,刘桂珍,196869.80,197041.04,2019-11-28 17:43:34,2019-01-03 18:57:21,非嫌疑人/亲属
12,6216913900145856,弓卫红,6662566.25,7735861.51,2020-01-17 18:22:48,2019-01-02 00:38:56,非嫌疑人/亲属
13,6216913901295049,毕丹丹,8289.94,8289.94,2019-10-22 06:27:40,2019-03-15 22:22:25,非嫌疑人/亲属
17,6226170101187595,解丽华,2120736.74,2104863.05,2019-12-21 01:00:32,2019-01-01 12:10:06,非嫌疑人/亲属
18,6226180992288881,郑婷婷,610002.03,620481.17,2020-01-07 05:16:58,2019-01-15 14:22:14,非嫌疑人/亲属
19,6226183900900001,陈月娥,82302914.19,79524618.87,2020-01-17 11:03:35,2019-01-01 00:23:56,非嫌疑人/亲属
21,6226190900991997,官侠,7751583.96,7476155.89,2020-01-14 19:47:53,2019-01-02 19:14:35,非嫌疑人/亲属
22,6226193900031475,武利军,320.36,135.01,2019-04-06 05:36:31,2019-01-11 03:48:31,非嫌疑人/亲属
